<a href="https://colab.research.google.com/github/vhppacheco/ic-ita-finance-ia/blob/main/clustering/Portfolio_Optimization_MV_IVP_HRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libs

In [ ]:
!pip install yfinance
!pip install -q alpha-vantage
!pip install -q ffn
!pip install -q config

# Para corrgir o bug: AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'
!pip install git+https://github.com/quantopian/pyfolio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import yfinance as yf
import pandas as pd
import scipy.cluster.hierarchy as sch
import numpy as np
import statsmodels.api as sm
from scipy import stats
import pandas as pd
from datetime import date
from matplotlib import pyplot as plt
import seaborn as sns
import cvxopt as opt
from cvxopt import blas, solvers
from alpha_vantage.timeseries import TimeSeries
import ffn
import config
import plotly.graph_objs as go
import plotly.express as pl
from plotly.subplots import make_subplots
import pyfolio as pf

%matplotlib inline
plt.close('all')

import warnings
warnings.filterwarnings('ignore')

#Lopez de Prado Algorithms

In [ ]:
# On 20151227 by MLdP <lopezdeprado@lbl.gov>
# Hierarchical Risk Parity


def getIVP(cov, **kargs):
    # Compute the inverse-variance portfolio
    ivp = 1. / np.diag(cov)
    ivp /= ivp.sum()
    return ivp


def getClusterVar(cov,cItems):
    # Compute variance per cluster
    cov_=cov.loc[cItems,cItems] # matrix slice
    w_=getIVP(cov_).reshape(-1,1)
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar


def getQuasiDiag(link):
    # Sort clustered items by distance
    link = link.astype(int)
    sortIx = pd.Series([link[-1, 0], link[-1, 1]])
    numItems = link[-1, 3]  # number of original items
    while sortIx.max() >= numItems:
        sortIx.index = range(0, sortIx.shape[0] * 2, 2)  # make space
        df0 = sortIx[sortIx >= numItems]  # find clusters
        i = df0.index
        j = df0.values - numItems
        sortIx[i] = link[j, 0]  # item 1
        df0 = pd.Series(link[j, 1], index=i + 1)
        sortIx = sortIx.append(df0)  # item 2
        sortIx = sortIx.sort_index()  # re-sort
        sortIx.index = range(sortIx.shape[0])  # re-index
    return sortIx.tolist()


def getRecBipart(cov, sortIx):
    # Compute HRP alloc
    w = pd.Series(1, index=sortIx)
    cItems = [sortIx]  # initialize all items in one cluster
    while len(cItems) > 0:
        cItems = [i[j:k] for i in cItems for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) if len(i) > 1]  # bi-section
        for i in range(0, len(cItems), 2):  # parse in pairs
            cItems0 = cItems[i]  # cluster 1
            cItems1 = cItems[i + 1]  # cluster 2
            cVar0 = getClusterVar(cov, cItems0)
            cVar1 = getClusterVar(cov, cItems1)
            alpha = 1 - cVar0 / (cVar0 + cVar1)
            w[cItems0] *= alpha  # weight 1
            w[cItems1] *= 1 - alpha  # weight 2
    return w


def correlDist(corr):
    # A distance matrix based on correlation, where 0<=d[i,j]<=1
    # This is a proper distance metric
    dist = ((1 - corr) / 2.)**.5  # distance matrix
    return dist


def getHRP(cov, corr):
    # Construct a hierarchical portfolio
    dist = correlDist(corr)
    link = sch.linkage(dist, 'single')
    dn = sch.dendrogram(link, labels=cov.index.values)
    plt.show(dn)
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()
    hrp = getRecBipart(cov, sortIx)
    return hrp.sort_index()

In [ ]:
def getMVP(cov):

    cov = cov.T.values
    n = len(cov)
    N = 100
    mus = [10 ** (5.0 * t / N - 1.0) for t in range(N)]

    # Convert to cvxopt matrices
    S = opt.matrix(cov)
    #pbar = opt.matrix(np.mean(returns, axis=1))
    pbar = opt.matrix(np.ones(cov.shape[0]))

    # Create constraint matrices
    G = -opt.matrix(np.eye(n))  # negative n x n identity matrix
    h = opt.matrix(0.0, (n, 1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu * S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S * x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']

    return list(wt)

In [ ]:
def get_all_portfolios(returns):
    
    cov, corr = returns.cov(), returns.corr()
    hrp = getHRP(cov, corr)
    ivp = getIVP(cov)
    ivp = pd.Series(ivp, index=cov.index)
    mvp = getMVP(cov)
    mvp = pd.Series(mvp, index=cov.index)
    
    portfolios = pd.DataFrame([mvp, ivp, hrp], index=['MVP', 'IVP', 'HRP']).T.sort_values(by='HRP',ascending=False)
    
    return portfolios

#Pre-selection 1: Sharpe Ratio

##Import Yahoo Finance

In [ ]:
prices= pd.DataFrame()

ativos = ['PRIO3.SA','EMBR3.SA','BEEF3.SA','HD','FANG','MRO','MRFG3.SA','JBSS3.SA','CPLE6.SA','INTU','CRL','TGT','ADBE','ALPA4.SA','^BVSP','^GSPC']

inicio = '2016-01-01'
fim = '2021-12-31'

for ativo in ativos:
  prices[ativo] = yf.download(ativo, start = inicio, end = fim)['Adj Close']
  prices.columns = prices.columns.str.rstrip(".SA")

df = prices.iloc[:, :-2]

#Benchmarks
bench = prices.iloc[:,-2:]
bench = bench/bench.iloc[0]

bench = (bench.pct_change(axis=0)).apply(np.log1p)
bench = bench.iloc[1:,:].rename(columns={'^BVSP':'Ibovespa','^GSPC':'S&P500'})
bench_acum = (1+bench).cumprod()

In [ ]:
df

### Normalize Data

In [ ]:
df = df/df.iloc[0]
df

### Log Returns

In [ ]:
log_returns = (df.pct_change(axis=0)).apply(np.log1p)
log_returns= log_returns.iloc[1:,:]
log_returns

###Correlation Matrix

In [ ]:
correlacao = log_returns.corr()
correlacao

In [ ]:
plt.figure(figsize = (20, 15))
sns.heatmap(round(correlacao,2), annot = True, cmap = "Blues")

### Covariance Matrix

In [ ]:
covariancia = log_returns.cov()
covariancia

In [ ]:
plt.figure(figsize = (20, 15))
sns.heatmap(covariancia, annot = True, cmap = 'Reds')

### Tree Clustering

In [ ]:
tree = getHRP(covariancia, correlacao)
tree

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform

plt.figure(figsize=(25,7))
distances = np.sqrt((1 - correlacao) / 2)
clusters = linkage(squareform(distances), method="ward")

dendrogram(clusters, labels=df.columns);

## Portfolios Optimazed

In [ ]:
portfolios = get_all_portfolios(log_returns)
portfolios

###Join Dataframes - Sectors

In [ ]:
df_cluster = pd.read_excel('/content/drive/MyDrive/IC ITA/Projeto de Pesquisa - IC ITA/Resultados/Ativos melhor cluster.xlsx').set_index('Código')
df_cluster

In [ ]:
portfolios = portfolios.merge(df_cluster, left_index = True, right_index = True)
portfolios = portfolios[['MVP','IVP','HRP','Setor NAICS','Indice']]
portfolios

#### EDA

In [ ]:
sum_sector = pd.DataFrame(portfolios.groupby(portfolios['Setor NAICS']).sum())
sum_sector

In [ ]:
def build_viz(coluna):

  y= round(sum_sector[coluna],3).sort_values(ascending=False)
  x = sum_sector.index

  fig = go.Figure(data=[go.Bar(x = x,
              y=y,
              text=y,
              textposition='auto',
              marker_color = 'blue'
          )])
  fig.update_layout(height=500, width=1200,title_text=coluna)
  fig.show()

In [ ]:
for i in sum_sector.columns:
  build_viz(i)

In [ ]:
def build_viz_2(coluna):
  fg = pl.sunburst(data_frame = sum_sector*100, path=[sum_sector.index], values = coluna ,title = coluna, height=500, width=1200)
  fg.update_traces(textfont_color = 'white',
                   textfont_size = 14,
                   hovertemplate = '<b>%{label}:</b> %{value:.2f}%')
  fg.show()

In [ ]:
for i in sum_sector.columns:
  build_viz_2(i)

### Combine returns and weights in Portfolios

In [ ]:
def combine_returns_weights (methods):

  portfolio_order = portfolios.T.iloc[:-2].reindex(columns=log_returns.columns)
  ativos = np.array(log_returns.columns)

  method = [methods]
  portfolio_methods = pd.DataFrame()

  for i in methods:

    pesos = np.array(portfolio_order.loc[i])
    carteira = pd.DataFrame((log_returns * pesos).sum(axis=1)).rename(columns={0:f'Retornos {i}'})

    #Retorno Total da Carteira
    total_return = (((1+carteira).cumprod())-1)*100
    print(f"O retorno total na carteira {i} é igual a " + f"{np.sum(total_return.iloc[-1,:]).round(2)}%")

    #Retorno Anual Esperado
    expected_return_annual = np.sum(carteira.mean()*252)*100
    print(f"O retorno anual na carteira {i} é igual a " + f"{expected_return_annual.round(2)}%")

    #Volatilidade Anual
    carteira_vol = np.sqrt(pesos.T.dot(covariancia*252).dot(pesos))*100
    print(f"A volatilidade anual da carteira {i} é igual a " + f"{carteira_vol.round(2)}%")

    #Sharpe das Carteiras
    sharpe=(expected_return_annual)/carteira_vol #sem considerar o risk free
    print(f"O Sharpe Ratio da carteira {i} é igual a " + f"{sharpe.round(3)}")

    #Max Drawdown
    carteira_acum = (1+carteira).cumprod()
    pico = carteira_acum.expanding(min_periods=1).max()
    dd = (carteira_acum/pico)-1
    drawdown = np.sum(dd.min())*100
    print(f"O MDD da carteira {i} é igual a " + f"{drawdown.round(2)}%")

    #Beta
    Y = carteira.values
    X = bench['Ibovespa'].values
    Z = bench['S&P500'].values

    X = sm.add_constant(X)
    Z = sm.add_constant(Z)

    modelo1 = sm.OLS(Y,X)
    resultado1 = modelo1.fit()
    beta_carteira_ibov = resultado1.params[1]
    beta_carteira_ibov = np.sum(beta_carteira_ibov)

    modelo2 = sm.OLS(Y,Z)
    resultado2 = modelo2.fit()
    beta_carteira_sp500 = resultado2.params[1]
    beta_carteira_sp500 = np.sum(beta_carteira_sp500)

    print(f"O Beta da carteira {i} em relação ao Ibovespa é igual a " + f"{beta_carteira_ibov.round(4)}\nE em relaçao ao S&P500 é igual a " + f"{beta_carteira_sp500.round(4)} \n")

    #Plots
    fig = plt.figure(1)
    plt.subplot(1,3,1)
    pf.plot_annual_returns(carteira)
    pf.plot_annual_returns(carteira)
    plt.subplot(1,3,2)
    pf.plot_monthly_returns_heatmap(carteira)
    plt.subplot(1,3,3)
    pf.plot_return_quantiles(carteira)
    plt.tight_layout()
    fig.set_size_inches(20,5)
    plt.show()

    #Plots
    plt.figure(figsize = (21.54,5))
    pf.plot_drawdown_underwater(carteira);
    plt.show()

    print("\n")

    portfolio_temp = carteira

    if portfolio_methods.empty == True:
      portfolio_methods = portfolio_temp

    else:
      portfolio_methods = pd.merge(portfolio_methods,portfolio_temp,left_index = True, right_index = True)
      portfolio_methods= pd.DataFrame(portfolio_methods)

  #Drawdown 30 dias 
  portfolios_methods['MDD 30d MVP'] = portfolios_methods['Retornos MVP'].rolling(window=30).min()
  portfolios_methods['MDD 30d IVP'] = portfolios_methods['Retornos IVP'].rolling(window=30).min()
  portfolios_methods['MDD 30d HRP'] = portfolios_methods['Retornos HRP'].rolling(window=30).min()

  #Retornos Acumulados
  portfolios_methods_acum = (1+portfolios_methods).cumprod()
  portfolios_methods_acum.iloc[0] = 1
  portfolios_methods_acum = pd.DataFrame(portfolios_methods_acum)

  #Join with Benchmarks
  portfolios_methods_acum = pd.merge(portfolios_methods_acum,bench_acum,left_index = True, right_index = True)

  ##Retorno Total Benchmarks
  ibov = portfolios_methods_acum.loc[portfolios_methods_acum.index[-1], "Ibovespa"]
  ibov = (ibov-1)*100
  sp500 = portfolios_methods_acum.loc[portfolios_methods_acum.index[-1], "S&P500"]
  sp500 = (sp500-1)*100
  print(f"O retorno total do Ibovespa é igual a " + f"{np.sum(ibov).round(2)}%")
  print(f"O retorno total do S&P500 é igual a " + f"{np.sum(sp500).round(2)}%")

  plot = portfolios_methods_acum[['Retornos MVP','Retornos IVP','Retornos HRP','Ibovespa','S&P500']].plot(figsize = (20,10));

  plot2 = portfolios_methods[['MDD 30d MVP','MDD 30d IVP','MDD 30d HRP']].plot(figsize = (19.74,10));

  individual_returns_anualized=log_returns.mean()*252
  individual_returns_anualized= individual_returns_anualized.to_frame()
  individual_returns_anualized.rename(columns={0:"Anual Returns %"}).sort_values(by = "Anual Returns %", ascending=False)

  return portfolio_methods, portfolios_methods_acum, plot, plot2, individual_returns_anualized

In [ ]:
portfolio, portfolios_acum, plot, plot2, individual_returns_anualized = combine_returns_weights(['MVP','IVP','HRP'])
portfolios_acum

In [ ]:
individual_returns_anualized

In [ ]:
teste = portfolios_methods.merge(bench,left_index=True,right_index=True)

In [ ]:
pf.create_full_tear_sheet(teste["Retornos MVP"], benchmark_rets=teste["Ibovespa"])

In [ ]:
pf.create_full_tear_sheet(teste["Retornos IVP"], benchmark_rets=teste["Ibovespa"])

In [ ]:
pf.create_full_tear_sheet(teste["Retornos HRP"], benchmark_rets=teste["Ibovespa"])

In [ ]:
pf.create_full_tear_sheet(teste["Retornos MVP"], benchmark_rets=teste["S&P500"])

In [ ]:
pf.create_full_tear_sheet(teste["Retornos IVP"], benchmark_rets=teste["S&P500"])

In [ ]:
pf.create_full_tear_sheet(teste["Retornos HRP"], benchmark_rets=teste["S&P500"])

##Secundary tests

In [ ]:
portfolio_order = portfolios.T.iloc[:-2].reindex(columns=log_returns.columns)
portfolio_order

In [ ]:
ativos = np.array(log_returns.columns)
pesos_mvp = np.array(portfolio_order.loc['MVP'])
pesos_ivp = np.array(portfolio_order.loc['IVP'])
pesos_hrp = np.array(portfolio_order.loc['HRP'])

In [ ]:
carteira_mvp = pd.DataFrame((log_returns * pesos_mvp).sum(axis=1)).rename(columns={0:'Retornos MVP'})
carteira_ivp = pd.DataFrame((log_returns * pesos_ivp).sum(axis=1)).rename(columns={0:'Retornos IVP'})
carteira_hrp = pd.DataFrame((log_returns * pesos_hrp).sum(axis=1)).rename(columns={0:'Retornos HRP'})

In [ ]:
carteira_mvp_vol=np.sqrt(pesos_ivp.T.dot(covariancia).dot(pesos_ivp))
carteira_mvp_vol

In [ ]:
expected_return_annual_mvp = np.sum(carteira_mvp.mean()*252)
expected_return_annual_mvp

In [ ]:
individual_returns_anualized=log_returns.mean()*252
individual_returns_anualized=individual_returns_anualized.to_frame()
individual_returns_anualized.rename(columns={0:"Anual Returns %"}).sort_values(by = "Anual Returns %", ascending=False)

In [ ]:
individual_returns_anualized.iloc[-1,:]

In [ ]:
portfolios_methods = carteira_mvp.merge(carteira_ivp,left_index = True, right_index = True).merge(carteira_hrp,left_index = True, right_index = True)
portfolios_methods

In [ ]:
portfolios_methods_acum = (1+portfolios_methods).cumprod()
# portfolios_methods_acum.iloc[0] = 1
portfolios_methods_acum

In [ ]:
portfolios_methods_acum[['Retornos MVP','Retornos IVP','Retornos HRP']].plot(figsize = (20,10));

In [ ]:
#Drawdown 30 dias 

portfolios_methods['MDD 30d MVP'] = portfolios_methods['Retornos MVP'].rolling(window=30).min()
portfolios_methods['MDD 30d IVP'] = portfolios_methods['Retornos IVP'].rolling(window=30).min()
portfolios_methods['MDD 30d HRP'] = portfolios_methods['Retornos HRP'].rolling(window=30).min()

In [ ]:
portfolios_methods[['MDD 30d MVP','MDD 30d IVP','MDD 30d HRP']].plot(figsize = (20,10));

In [ ]:
bench

In [ ]:
bench_acum

In [ ]:
portfolios_methods_acum = portfolios_methods_acum.merge(bench_acum,left_index=True,right_index=True)
# portfolios_methods_acum.iloc[0] = 1
# portfolios_methods_acum = 10000 * portfolios_methods_acum
# portfolios_methods_acum['teste'] = portfolios_methods_acum["Retornos MVP"].pct_change()
# portfolios_methods_acum

In [ ]:
portfolios_methods_acum

In [ ]:
portfolios_methods_acum.plot(figsize = (20,10));

In [ ]:
teste = portfolios_methods.merge(bench,left_index=True,right_index=True)
teste

In [ ]:
pf.create_full_tear_sheet(teste["Retornos MVP"], benchmark_rets=teste["Ibovespa"])

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,8))
pf.plot_rolling_volatility(teste["Retornos HRP"], factor_returns=teste["^S&P500"], ax=ax1);

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,8))
pf.plot_rolling_beta(teste["Retornos HRP"], factor_returns=teste["S&P500"], ax=ax1)
plt.ylim((0.6, 1.5));

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,8))
pf.plot_monthly_returns_heatmap(teste["Retornos HRP"], ax=ax1)

In [ ]:
pf.create_bayesian_tear_sheet(teste['Retornos MVP'], live_start_date='2016-01-05')

In [ ]:
[f for f in dir(pf.plotting) if 'plot_' in f]

In [ ]:
pf.timeseries.sharpe_ratio(teste['Retornos MVP'])